In [1]:
import pandas as pd
import numpy as np
import bioframe
import multiprocessing as mp
import glob

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
interval_size = 1_000  # bigwigs will have this resolution as interval size

chromsizes = bioframe.fetch_chromsizes("hg38")[:23]
NPROC = 18

bins_df = []
for chrom, size in chromsizes.items():
    bins_df.append(
        pd.DataFrame(
            {
                "chrom": chrom,
                "start": np.arange(0, size // interval_size * interval_size, interval_size),
                "end": np.arange(interval_size, (size + interval_size) // interval_size * interval_size, interval_size),
                "count": 0,
            }
        )
    )

bins_df = pd.concat(
    bins_df
)  # an empty dataframe with all the bins as coordinates, this will have the counts added to it

bins_df.reset_index(drop=True, inplace=True)

In [3]:
blacklist_path = "/cta/users/vkaya/hi-c/work/hela/notebooks/repair-data/hg38_blacklist.bed"

In [4]:
class xr_ds_exp:
    def __init__(self, file_path, blacklist=None):
        self.file_path = file_path
        if "sim" in file_path:
            self.sim = True
        else:
            self.sim = False

        if "64" in file_path:
            self.lesion_type = "64"
        elif "CPD" in file_path:
            self.lesion_type = "CPD"
        else:
            self.lesion_type = None

        if "plus" in file_path:
            self.strand = "plus"
        elif "minus" in file_path:
            self.strand = "minus"
        else:
            self.strand = None

        if "XR" in file_path:
            self.experiment = "XR"
        elif "DS" in file_path:
            self.experiment = "DS"
        else:
            self.experiment = None

        if 'rep1' in file_path:
            self.rep = 'rep1'
        elif 'rep2' in file_path:
            self.rep = 'rep2'
        else:
            self.rep = None

        self.df = pd.read_csv(file_path, sep="\t", header=None)
        self.df.columns = ["chrom", "start", "end", "name", "read_length", "strand"]

        self.exact_sites = False

        if blacklist is not None:
            blacklist_df = bioframe.read_table(blacklist).iloc[:, :3]
            blacklist_df.columns = ['chrom', 'start', 'end']
            self.blacklist_df = blacklist_df
            self.filter_blacklist()

    def intersect(self, bins_df, chromNames, interval_size):
        df = self.df.copy()
        all_bins_df = []
        for chrName, chrDF in (
            df.loc[df.chrom.isin(chromNames)].groupby(["chrom"]).__iter__()
        ):
            bins_df_curr = (
                bins_df.copy().loc[bins_df.chrom == chrName].reset_index(drop=True)
            )
            start_pos = chrDF.start
            bins = start_pos // interval_size
            counts = bins.groupby(bins).count()
            bins_df_curr.loc[counts.index, "count"] = counts.values
            all_bins_df.append(bins_df_curr)

        self.df_intersect = pd.concat(all_bins_df)

    def get_exact_sites(self):
        if self.exact_sites == False:
            if self.experiment == "XR" and self.strand == "plus":
                self.df.start = self.df.end - 8
                self.df.end = self.df.end - 6
                self.exact_sites = True
            elif self.experiment == "XR" and self.strand == "minus":
                self.df.start = self.df.start + 6
                self.df.end = self.df.start + 8
                self.exact_sites = True
            elif self.experiment == "DS" and self.strand == "plus":
                self.df.start = self.df.end - 5
                self.df.end = self.df.end - 3
                self.exact_sites = True
            elif self.experiment == "DS" and self.strand == "minus":
                self.df.start = self.df.start + 3
                self.df.end = self.df.start + 5
                self.exact_sites = True
            self.df.sort_values(by=["chrom", "start", "end"], inplace=True)
        else:
            print(f"exact_sites {self.exact_sites}")

    def filter_blacklist(self):
        if self.blacklist_df is not None:
            self.df = bioframe.ops.subtract(self.df, self.blacklist_df)

In [5]:
def intersect_wrapper(file_in, bins_df, chromNames, blacklist_path, interval_size):
    exp = xr_ds_exp(file_in, blacklist_path)
    exp.get_exact_sites()
    exp.intersect(bins_df, chromNames, interval_size)
    return exp

In [6]:
chromNames = chromsizes.keys()
path = "/cta/users/vkaya/rep_dmg_snakemake/dedup/xr-ds-seq-snakemake/results/processed_files/*.bed"
args = [(file_in, bins_df, chromNames, blacklist_path, interval_size) for file_in in glob.glob(path)]
with mp.Pool(NPROC) as pool:
    results = pool.starmap(intersect_wrapper, args)

In [39]:
def get_name(exp):
    return (
        f"{exp.experiment}_{exp.lesion_type}_{exp.strand}_{exp.rep}_sim"
        if exp.sim
        else f"{exp.experiment}_{exp.lesion_type}_{exp.strand}_{exp.rep}_real"
    )

In [40]:
## merge plus and minus strands

plus_minus_merged = {}

prev = []
for res_1 in results:
    for res_2 in results:
        name_1 = get_name(res_1)
        name_2 = get_name(res_2)

        if set((name_1, name_2)) in prev:
            continue
        elif name_1 == name_2:
            continue
        else:
            prev.append(set((name_1, name_2)))

            if (
                res_1.experiment == res_2.experiment
                and res_1.lesion_type == res_2.lesion_type
                and res_1.sim == res_2.sim
                and res_1.rep == res_2.rep
                and res_1.strand != res_2.strand
            ):
                key_name = (
                    f"{res_1.experiment}_{res_1.lesion_type}_{res_1.rep}_sim"
                    if res_1.sim
                    else f"{res_1.experiment}_{res_1.lesion_type}_{res_1.rep}_real"
                )
                print(f"Processing {name_1} and {name_2}")
                print(f"Key name: {key_name}")
                intersect_1 = res_1.df_intersect.copy()
                intersect_2 = res_2.df_intersect.copy()
                assert (
                    intersect_1["chrom"].values.tolist()
                    == intersect_2["chrom"].values.tolist()
                )
                counts_1 = intersect_1["count"].values
                counts_2 = intersect_2["count"].values
                counts = counts_1 + counts_2
                intersect_1["count"] = counts

                plus_minus_merged[key_name] = intersect_1

Processing XR_CPD_minus_rep1_sim and XR_CPD_plus_rep1_sim
Key name: XR_CPD_rep1_sim
Processing DS_CPD_plus_rep1_real and DS_CPD_minus_rep1_real
Key name: DS_CPD_rep1_real
Processing XR_CPD_minus_rep1_real and XR_CPD_plus_rep1_real
Key name: XR_CPD_rep1_real
Processing XR_64_minus_rep2_sim and XR_64_plus_rep2_sim
Key name: XR_64_rep2_sim
Processing DS_CPD_minus_rep1_sim and DS_CPD_plus_rep1_sim
Key name: DS_CPD_rep1_sim
Processing DS_CPD_plus_rep2_real and DS_CPD_minus_rep2_real
Key name: DS_CPD_rep2_real
Processing XR_64_minus_rep1_real and XR_64_plus_rep1_real
Key name: XR_64_rep1_real
Processing DS_64_plus_rep2_real and DS_64_minus_rep2_real
Key name: DS_64_rep2_real
Processing DS_64_plus_rep1_sim and DS_64_minus_rep1_sim
Key name: DS_64_rep1_sim
Processing XR_64_plus_rep1_sim and XR_64_minus_rep1_sim
Key name: XR_64_rep1_sim
Processing DS_64_plus_rep1_real and DS_64_minus_rep1_real
Key name: DS_64_rep1_real
Processing DS_64_plus_rep2_sim and DS_64_minus_rep2_sim
Key name: DS_64_rep2

In [41]:
## merge rep1 and rep2

rep1_rep2_merged = {}

prev = []
for k_1 in plus_minus_merged:
    for k_2 in plus_minus_merged:
        if set((k_1, k_2)) in prev:
            continue
        elif k_1 == k_2:
            continue
        else:
            prev.append(set((k_1, k_2)))

            if (
                k_1.split("_")[0] == k_2.split("_")[0]
                and k_1.split("_")[1] == k_2.split("_")[1]
                and k_1.split("_")[2] != k_2.split("_")[2]
                and k_1.split("_")[3] == k_2.split("_")[3]
            ):
                key_name = "_".join([l for i,l in enumerate(k_1.split("_")) if i != 2])
                print(f"Processing {k_1} and {k_2}")
                print(f"Key name: {key_name}")
                intersect_1 = plus_minus_merged[k_1].copy()
                intersect_2 = plus_minus_merged[k_2].copy()
                assert (
                    intersect_1["chrom"].values.tolist()
                    == intersect_2["chrom"].values.tolist()
                )
                counts_1 = intersect_1["count"].values
                counts_2 = intersect_2["count"].values
                counts = counts_1 + counts_2
                intersect_1["count"] = counts
                rep1_rep2_merged[key_name] = intersect_1

Processing XR_CPD_rep1_sim and XR_CPD_rep2_sim
Key name: XR_CPD_sim
Processing DS_CPD_rep1_real and DS_CPD_rep2_real
Key name: DS_CPD_real
Processing XR_CPD_rep1_real and XR_CPD_rep2_real
Key name: XR_CPD_real
Processing XR_64_rep2_sim and XR_64_rep1_sim
Key name: XR_64_sim
Processing DS_CPD_rep1_sim and DS_CPD_rep2_sim
Key name: DS_CPD_sim
Processing XR_64_rep1_real and XR_64_rep2_real
Key name: XR_64_real
Processing DS_64_rep2_real and DS_64_rep1_real
Key name: DS_64_real
Processing DS_64_rep1_sim and DS_64_rep2_sim
Key name: DS_64_sim


In [42]:
from scipy.stats import gmean
def normalize(counts_df, add_pseudocount=None):
    counts = counts_df["count"]
    print(f"Total counts: {counts.sum()}")
    if add_pseudocount != None:
        if type(add_pseudocount) == float:
            counts += add_pseudocount
            print(f"Added {add_pseudocount} pseudocount")
        elif add_pseudocount == "mean":
            pseudo_count = counts.mean()
            counts += pseudo_count
            print(f"Added {pseudo_count} pseudocount")
        elif add_pseudocount == "base_level":
            pseudo_count = counts.sum() / len(counts.loc[counts > 0])
            counts += pseudo_count
            print(f"Added {pseudo_count} pseudocount")
    counts_df["count"] = counts * 1_000_000 / counts.sum()
    return counts_df

normalized = {}
for k, v in rep1_rep2_merged.items():
    print(f"Normalizing {k}")
    normalized[k] = normalize(v.copy(), add_pseudocount = None)

Normalizing XR_CPD_sim
Total counts: 26116696
Normalizing DS_CPD_real
Total counts: 16822127
Normalizing XR_CPD_real
Total counts: 26452698
Normalizing XR_64_sim
Total counts: 35161521
Normalizing DS_CPD_sim
Total counts: 15556897
Normalizing XR_64_real
Total counts: 35619658
Normalizing DS_64_real
Total counts: 18575746
Normalizing DS_64_sim
Total counts: 17106264


In [43]:
for k, v in normalized.items():
    print(
        k,
        v["count"].mean(),
        v["count"].median(),
        v["count"].std(),
        v["count"].sum(),
        v["count"].min(),
        v["count"].max(),
    )

XR_CPD_sim 0.3299208519876079 0.3063174606772618 0.2000516621851465 999999.9999999992 0.0 4.2118650843123495
DS_CPD_real 0.3299208519876081 0.2972275741349474 0.23895905999575145 999999.9999999998 0.0 4.696195671332169
XR_CPD_real 0.32992085198760834 0.2268199636951966 0.3764337262729941 1000000.0000000005 0.0 11.681228130302625
XR_64_sim 0.3299208519876081 0.31284198428162424 0.21079707674506215 999999.9999999999 0.0 3.7825439917687294
DS_CPD_sim 0.3299208519876083 0.32140085519625156 0.20655582040402193 1000000.0000000003 0.0 41.846391346551954
XR_64_real 0.3299208519876082 0.30881823739015124 0.25516259424220267 1000000.0 0.0 4.688422331286842
DS_64_real 0.3299208519876083 0.3230018326047309 0.21848100059282238 1000000.0000000003 0.0 5.060362044140785
DS_64_sim 0.32992085198760807 0.35074870819250775 0.17696543035162177 999999.9999999997 0.0 5.319688740919701


In [44]:
def clip_std(df, n_std=3):
    arr = df['count'].copy()
    arr = np.clip(arr, -n_std * np.nanstd(arr) + np.mean(arr), n_std * np.nanstd(arr) + np.mean(arr))
    df['count'] = np.where(np.isnan(arr), np.nan, arr)
    return df

from sklearn.preprocessing import QuantileTransformer
def qt(df):
    arr = df['count'].copy()
    arr = arr.replace([np.inf, -np.inf], np.nan)
    arr = np.where(np.isnan(arr), np.nan, arr)
    arr = arr.reshape(-1, 1)
    qt = QuantileTransformer(n_quantiles=1000, output_distribution='uniform')
    arr = qt.fit_transform(arr)
    df['count'] = arr * 2
    return df

from sklearn.preprocessing import StandardScaler
def zscore(df):
    arr = df['count'].copy()
    arr = arr.replace([np.inf, -np.inf], np.nan)
    arr = np.where(np.isnan(arr), np.nan, arr)
    arr = arr.reshape(-1, 1)
    scaler = StandardScaler()
    arr = scaler.fit_transform(arr)
    df['count'] = arr
    return df

In [45]:
over_sim = {}

over_sim["XR_CPD_real_over_sim"] = normalized["XR_CPD_real"].copy()
over_sim["XR_CPD_real_over_sim"]["count"] /= normalized["XR_CPD_sim"]["count"]

over_sim["XR_64_real_over_sim"] = normalized["XR_64_real"].copy()
over_sim["XR_64_real_over_sim"]["count"] /= normalized["XR_64_sim"]["count"]

over_sim["DS_CPD_real_over_sim"] = normalized["DS_CPD_real"].copy()
over_sim["DS_CPD_real_over_sim"]["count"] /= normalized["DS_CPD_sim"]["count"]

over_sim["DS_64_real_over_sim"] = normalized["DS_64_real"].copy()
over_sim["DS_64_real_over_sim"]["count"] /= normalized["DS_64_sim"]["count"]

over_sim["XR_CPD_rep_eff"] = over_sim["XR_CPD_real_over_sim"].copy()
over_sim["XR_CPD_rep_eff"]["count"] /= over_sim["DS_CPD_real_over_sim"]["count"]

over_sim["XR_64_rep_eff"] = over_sim["XR_64_real_over_sim"].copy()
over_sim["XR_64_rep_eff"]["count"] /= over_sim["DS_64_real_over_sim"]["count"]

In [46]:
for k, v in over_sim.items():
    #over_sim[k] = qt(clip_std(v.reset_index(drop=True).replace([np.inf, -np.inf], np.nan)))
    #over_sim[k] = clip_std(v.reset_index(drop=True).replace([np.inf, -np.inf], np.nan), 3)
    over_sim[k] = v.reset_index(drop=True).replace([np.inf, -np.inf], np.nan)



for k, v in over_sim.items():
    print(
        k,
        v["count"].mean(),
        v["count"].median(),
        v["count"].std(),
        v["count"].sum(),
        v["count"].min(),
        v["count"].max(),
    )
    a, b = pd.qcut(v["count"], 4, retbins=True)
    print(b)

XR_CPD_real_over_sim 1.182501818395577 0.7678984494427668 1.6456957689855902 3241411.311989581 0.0 100.70439665549428
[  0.           0.37973      0.76789845   1.41042572 100.70439666]
XR_64_real_over_sim 1.1320465635344819 0.9562900202115923 0.952388127834937 3112935.601804024 0.0 60.2154232081622
[ 0.          0.59228285  0.95629002  1.419011   60.21542321]
DS_CPD_real_over_sim 1.2889167795760257 0.9247877512754481 1.2824212858323185 3577641.149402056 0.0 36.5291161753802
[ 0.          0.55487265  0.92478775  1.61837856 36.52911618]
DS_64_real_over_sim 1.2508587221100007 0.9208924368367225 1.1746015595479498 3497988.8906189534 0.0 29.468557978775117
[ 0.          0.57555777  0.92089244  1.53482073 29.46855798]
XR_CPD_rep_eff 1.5661950273544312 0.7371483515021189 3.356393665850783 4133003.866175116 0.0 617.221940385326
[0.00000000e+00 3.11381631e-01 7.37148352e-01 1.64714528e+00
 6.17221940e+02]
XR_64_rep_eff 1.5001647657629529 0.9095217587887839 2.320224109814775 3992888.0459919455 0

In [47]:
out_path = "/cta/users/vkaya/hi-c/work/hela/notebooks/repair-data/bws_bioframe"

with mp.Pool(8) as pool:
    pool.starmap(
        bioframe.to_bigwig,
        [
            (v, chromsizes, f"{out_path}/{k}_res{interval_size}.bw", "count")
            for k, v in over_sim.items()
        ],
    )